In [0]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
CAL_DTYPES={
    "event_name_1": "category", 
    "event_name_2": "category", 
    "event_type_1": "category", 
    "event_type_2": "category", 
    "weekday": "category", 
    "wm_yr_wk": "int16", 
    "wday": "int16",
    "month": "int16", 
    "year": "int16", 
    "snap_CA": "float32", 
    "snap_TX": "float32", 
    "snap_WI": "float32" 
}
PRICE_DTYPES = {
    "store_id": "category",
     "item_id": "category", 
     "wm_yr_wk": "int16",
     "sell_price":"float32"
}

In [0]:
DATA_DIR = '/gdrive/My Drive/M5-forecasting/'
h = 28 
BACKWARD_LAGS = 60
END_D = 1913
END_DATE = datetime(2016, 4, 25) 
np.random.seed(0)

In [0]:
def create_dt(is_train = True, nrows = None, first_day = 1500):
    prices = pd.read_csv(DATA_DIR+"sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(DATA_DIR+"calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])

    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else END_D-BACKWARD_LAGS, first_day)
    numcols = [f"d_{day}" for day in range(start_day, END_D+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv(DATA_DIR+"sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(END_D +1, END_D + 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [0]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [0]:
FIRST_DAY = 1400

In [0]:
%%time
df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 15.3 s, sys: 3.5 s, total: 18.8 s
Wall time: 22.3 s


In [0]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,8.26
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,3.97
2,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,2.97
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1400,3.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,4.64
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,2.88


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15537606 entries, 0 to 15537605
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [0]:
%%time
create_fea(df)
df.shape

CPU times: user 2min 3s, sys: 2.16 s, total: 2min 5s
Wall time: 2min 5s


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15537606 entries, 0 to 15537605
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [0]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,48,4,28
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,3.97,NaN,NaN,NaN,NaN,NaN,NaN,48,4,28
2,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,2.97,NaN,NaN,NaN,NaN,NaN,NaN,48,4,28
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1400,3.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,4.64,NaN,NaN,NaN,NaN,NaN,NaN,48,4,28
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0.0,0.0,0.0,2.88,NaN,NaN,NaN,NaN,NaN,NaN,48,4,28


In [0]:
df.dropna(inplace = True)
df.shape

(13860656, 31)

In [0]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [0]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)

CPU times: user 7.77 s, sys: 759 ms, total: 8.53 s
Wall time: 8.52 s


In [0]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

54

In [0]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [0]:
%%time

m_lgb = lgb.train(params, 
                  train_data, 
                  valid_sets = [fake_valid_data], 
                  verbose_eval=20) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[20]	valid_0's rmse: 2.48775
[40]	valid_0's rmse: 2.2178
[60]	valid_0's rmse: 2.14983
[80]	valid_0's rmse: 2.13033
[100]	valid_0's rmse: 2.12059
[120]	valid_0's rmse: 2.11354
[140]	valid_0's rmse: 2.10844
[160]	valid_0's rmse: 2.10244
[180]	valid_0's rmse: 2.0966
[200]	valid_0's rmse: 2.09106
[220]	valid_0's rmse: 2.08791
[240]	valid_0's rmse: 2.08513
[260]	valid_0's rmse: 2.08301
[280]	valid_0's rmse: 2.07984
[300]	valid_0's rmse: 2.07798
[320]	valid_0's rmse: 2.07543
[340]	valid_0's rmse: 2.07371
[360]	valid_0's rmse: 2.07257
[380]	valid_0's rmse: 2.06983
[400]	valid_0's rmse: 2.06768
[420]	valid_0's rmse: 2.06607
[440]	valid_0's rmse: 2.06339
[460]	valid_0's rmse: 2.0616
[480]	valid_0's rmse: 2.06036
[500]	valid_0's rmse: 2.05916
[520]	valid_0's rmse: 2.05687
[540]	valid_0's rmse: 2.05527
[560]	valid_0's rmse: 2.05342
[580]	valid_0's rmse: 2.05213
[600]	valid_0's rmse: 2.05065
[620]	valid_0's rmse: 2.04926
[640]	valid_0's rmse: 2.04803
[660]	valid_0's rmse: 2.04658
[680]	valid_0's r

In [0]:
m_lgb.save_model("model.lgb")

In [0]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1 / len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = END_DATE + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=BACKWARD_LAGS)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha * m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= END_DATE, ["id", "sales"]].copy()

    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [0]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.694917,0.752471,0.754582,0.753150,1.052376,1.186456,1.054752,0.763473,0.707377,0.713147,0.724177,1.004002,1.294059,1.010077,0.870927,0.770666,0.845576,0.826878,1.056684,1.419039,1.236099,0.876952,0.761386,0.779081,0.770357,0.997343,1.328039,1.198868
1,FOODS_1_001_CA_2_validation,0.764737,0.713269,0.658067,0.674257,0.887919,1.108696,0.907761,0.792810,0.805210,0.878964,0.854828,1.081331,1.490971,1.289814,0.889266,0.892050,0.911307,0.882796,1.051971,1.646266,0.909664,0.536033,0.503229,0.520753,0.532442,0.632864,1.069100,1.316148
2,FOODS_1_001_CA_3_validation,0.661003,0.613899,0.594123,0.665871,0.813425,1.324989,1.546010,1.152552,1.140815,1.104877,1.199138,1.521508,2.068470,1.561306,1.124035,1.057675,1.094199,1.120317,1.407421,2.026644,1.326775,0.778623,0.728413,0.672761,0.681849,0.887917,1.356077,1.697919
3,FOODS_1_001_CA_4_validation,0.388224,0.321583,0.348039,0.351394,0.370363,0.413981,0.367513,0.324055,0.329643,0.360946,0.326811,0.359720,0.398738,0.354496,0.331931,0.330060,0.351615,0.349600,0.351391,0.408541,0.385211,0.339786,0.329840,0.332580,0.331580,0.367414,0.404537,0.384386
4,FOODS_1_001_TX_1_validation,0.179126,0.181186,0.179805,0.181161,0.195914,0.212531,0.185345,0.364740,0.370249,0.343376,0.324231,0.355530,0.399540,0.316039,0.315183,0.374008,0.358463,0.328644,0.337866,0.364967,0.310902,0.265099,0.260696,0.259064,0.257343,0.251513,0.306906,0.292824
5,FOODS_1_001_TX_2_validation,0.473003,0.426398,0.435832,0.437532,0.480572,0.580027,0.501993,0.402154,0.416045,0.419096,0.461589,0.544350,0.668884,0.479996,0.470675,0.447210,0.446552,0.435770,0.535869,0.650093,0.612640,0.447904,0.419228,0.417619,0.415926,0.514312,0.626598,0.582138
6,FOODS_1_001_TX_3_validation,0.404118,0.389440,0.401637,0.471192,0.494556,0.471899,0.516070,0.411579,0.478104,0.476410,0.468515,0.533824,0.590169,0.467668,0.460527,0.455735,0.469913,0.485964,0.520556,0.594127,0.566441,0.442224,0.434095,0.442065,0.454700,0.490229,0.548125,0.495672
7,FOODS_1_001_WI_1_validation,0.348886,0.414848,0.382348,0.396833,0.500632,0.506067,0.574207,0.441823,0.446245,0.443449,0.526317,0.633814,0.790295,0.674057,0.510092,0.513941,0.518447,0.567602,0.725142,0.932666,0.840185,0.541280,0.570683,0.563319,0.573363,0.796247,0.882532,0.808534
8,FOODS_1_001_WI_2_validation,0.318699,0.317227,0.343764,0.346648,0.419075,0.444170,0.456579,0.375717,0.466818,0.407641,0.398606,0.502345,0.530682,0.472202,0.455446,0.414022,0.485648,0.463045,0.504556,0.547674,0.515197,0.381918,0.383797,0.378677,0.383044,0.428828,0.455066,0.426740
9,FOODS_1_001_WI_3_validation,0.236639,0.242035,0.232483,0.235570,0.261549,0.315759,0.350083,0.304408,0.351831,0.344273,0.317347,0.381624,0.477818,0.376330,0.324247,0.333040,0.324783,0.311982,0.358331,0.507218,0.449146,0.303822,0.299970,0.288940,0.305645,0.351707,0.484699,0.403138


In [0]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.910026,0.847982,0.850890,0.801789,1.071719,1.296671,1.350886,1.009637,0.970509,0.995156,0.974088,1.114272,1.343018,1.194240,0.992385,0.929573,0.965841,0.963692,1.112975,1.385381,1.294539,0.993150,0.869451,0.849285,0.855230,1.022105,1.262562,1.233375
1,FOODS_1_001_CA_2_validation,0.940942,0.954666,0.882539,1.273918,1.290628,1.354433,1.554513,0.920688,0.926300,0.931731,0.960712,1.120798,1.493172,1.338206,1.020169,0.987166,1.002741,1.031221,1.208342,1.641095,1.501229,1.032052,0.982766,1.012417,1.052117,1.241158,1.689393,1.424921
2,FOODS_1_001_CA_3_validation,1.091795,1.044925,0.949970,0.917520,0.990185,1.107758,1.223882,1.105829,1.137468,1.004840,1.102019,1.126727,1.455431,1.267847,1.116070,1.112878,1.048881,1.069820,1.132724,1.674212,1.766982,1.147176,1.055445,0.980131,0.969123,1.054151,1.391111,1.266828
3,FOODS_1_001_CA_4_validation,0.414818,0.361572,0.355099,0.349345,0.405143,0.451183,0.517519,0.398493,0.426824,0.414165,0.423309,0.406780,0.444357,0.421732,0.376850,0.383107,0.405915,0.428294,0.457459,0.482851,0.500463,0.385219,0.363917,0.368164,0.373390,0.419760,0.457920,0.481183
4,FOODS_1_001_TX_1_validation,0.180371,0.179159,0.170700,0.171773,0.172805,0.180151,0.228228,0.476832,0.429286,0.425293,0.456359,0.464892,0.468873,0.397233,0.370246,0.397884,0.327414,0.328859,0.393753,0.380485,0.353399,0.295464,0.280412,0.275795,0.272560,0.278792,0.331847,0.318894
5,FOODS_1_001_TX_2_validation,0.482585,0.436249,0.445677,0.395896,0.447061,0.496326,0.577034,0.472376,0.479704,0.438499,0.524659,0.537258,0.614668,0.558372,0.494018,0.450336,0.495783,0.476332,0.553735,0.581341,0.591867,0.459162,0.425086,0.428849,0.435233,0.481264,0.547399,0.528644
6,FOODS_1_001_TX_3_validation,0.398578,0.368744,0.376498,0.413743,0.451219,0.457103,0.598738,0.482092,0.538495,0.483629,0.515115,0.534072,0.571413,0.489207,0.485516,0.451393,0.498438,0.498808,0.542449,0.567497,0.566333,0.453887,0.432316,0.435424,0.440499,0.471629,0.539260,0.506802
7,FOODS_1_001_WI_1_validation,0.331981,0.372827,0.364709,0.363885,0.447759,0.680631,0.649428,0.457657,0.437654,0.419107,0.512862,0.592773,0.799761,0.613822,0.536360,0.525176,0.572492,0.578808,0.714459,0.864306,0.765611,0.558961,0.541106,0.556651,0.567618,0.703742,1.026890,0.805103
8,FOODS_1_001_WI_2_validation,0.310635,0.340609,0.353715,0.338573,0.414242,0.423919,0.425699,0.437361,0.506364,0.413411,0.451346,0.507607,0.515267,0.403375,0.435103,0.417913,0.494655,0.484010,0.488920,0.529035,0.497015,0.364418,0.391413,0.381275,0.383275,0.403016,0.440113,0.405239
9,FOODS_1_001_WI_3_validation,0.237874,0.235062,0.209730,0.220884,0.267701,0.319444,0.379183,0.366106,0.388815,0.348863,0.381240,0.436294,0.422055,0.371011,0.346316,0.336610,0.364897,0.351345,0.413273,0.463644,0.426028,0.320039,0.311917,0.324671,0.319644,0.378388,0.424405,0.364692


In [0]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [0]:
sub.shape

(60980, 29)